In [ ]:
import re
import time
import requests

import pandas as pd
import numpy as np
import urllib.request as req
import matplotlib.pyplot as plt

from matplotlib import font_manager, rc
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib import parse
from ast import literal_eval

# 1. 카테고리-기업명-기업코드 연결하기

## 1) 카테고리-기업명 가져오기

In [ ]:
#네이버 증권의 업종별 분류 사이트 연결

driver = webdriver.Chrome('C:/pywork/chromedriver')
url2 = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'
driver.get(url2)

In [ ]:
event_category_company_list = list()

#1~15번째의 업종 카테고리 가져오기

for j in range(0, 15):
    for i in range(4+(8*j),4+(8*j)+5):
        driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr[{}'.format(i)+']/td[1]/a').click()

        #카테고리명 가져오기

        category_name = driver.find_elements_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr[4]/td[1]')[0].text
    
        #회사명 가져오기
        text_length_temp = driver.find_elements_by_xpath('//*[@id="contentarea"]/div[4]/table/tbody')[0].text
        text_length = text_length_temp.split('\n')
        event_company_raw_data_list = list()
        
        for i in range(1, len(text_length)//2):
               
            company_name_temp = driver.find_elements_by_xpath('//*[@id="contentarea"]/div[4]/table/tbody/tr[{}'.format(i)+']/td[1]')[0].text
            company_name = re.sub(r"[\s|*]","", company_name_temp)
            event_company_raw_data = company_name
            event_category_company_list += [[category_name, event_company_raw_data]]

        driver.find_element_by_xpath('//*[@id="contentarea"]/div[4]/div/div/a/img').click()

In [ ]:
#16번째의 업종 카테고리 가져오기
#16번째 업종의 경우 항목이 5개가 아닌 4개

for k in range(124,128):
    driver.find_element_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr[{}'.format(k)+']/td[1]/a').click()

    #카테고리명 가져오기

    category_name = driver.find_elements_by_xpath('//*[@id="contentarea_left"]/table/tbody/tr[4]/td[1]')[0].text
    
    #회사명 가져오기
    text_length_temp = driver.find_elements_by_xpath('//*[@id="contentarea"]/div[4]/table/tbody')[0].text
    text_length = text_length_temp.split('\n')
    event_company_raw_data_list = list()
        
    for i in range(1, len(text_length)//2):
        company_name_temp = driver.find_elements_by_xpath('//*[@id="contentarea"]/div[4]/table/tbody/tr[{}'.format(i)+']/td[1]')[0].text
        company_name = re.sub(r"[\s|*]","", company_name_temp)
        event_company_raw_data = company_name
        event_category_company_list += [[category_name, event_company_raw_data]]

    driver.find_element_by_xpath('//*[@id="contentarea"]/div[4]/div/div/a/img').click()

In [ ]:
#카테고리-기업명 데이터프레임

category_company_df = pd.DataFrame(event_category_company_list)
category_company_df.columns = ['카테고리', '기업명']

In [ ]:
#파일 저장
category_company_df.to_csv('c:/pywork/category_company.csv', encoding = 'euc-kr')

## 2) 기업명-기업코드 가져오기

- 한국거래소 제공 기업코드 활용하기
- http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201
- 전체항목의 csv 파일 다운로드

In [ ]:
# csv 불러오기
company_number_raw_data = pd.read_csv('c:/pywork/data_5905_20210724.csv', encoding = 'euc-kr')

In [ ]:
#기업코드 가공하기

company_number_temp_list = list(company_number_raw_data['단축코드'])


#단축코드를 기업코드 6자리로 변경
company_number_list = list()

for i in range(0, len(company_number_temp_list)):
    if len(company_number_temp_list[i]) == 6 :
        company_number_list += [company_number_temp_list[i]]
    elif len(company_number_temp_list[i]) == 5 :
        company_number_list += ['0'+company_number_temp_list[i]]
    elif len(company_number_temp_list[i]) == 4 :
        company_number_list += ['00'+company_number_temp_list[i]]
    elif len(company_number_temp_list[i]) == 3 :
        company_number_list += ['000'+company_number_temp_list[i]]
    elif len(company_number_temp_list[i]) == 2 :
        company_number_list += ['0000'+company_number_temp_list[i]]
    else:
        pass

In [ ]:
#기업명 가공하기

company_name_temp_list = list(company_number_raw_data['한글 종목약명'])


#글자 사이 공백 제거
company_name_list = list()

for i in range(0, len(company_name_temp_list)):
    company_name_list += [re.sub(r"[\s]","", company_name_temp_list[i])]

In [ ]:
#기업코드 컬럼 추가
company_number_raw_data['기업코드'] = company_number_list
company_number_raw_data['기업명'] = company_name_list

In [ ]:
#필요한 컬럼만 가져오기
company_number_df = company_number_raw_data[['기업명', '기업코드']]

In [ ]:
company_number_df.to_csv('c:/pywork/company_number.csv', encoding = 'euc-kr')

## 3) 카테고리-기업명-기업코드 합치기

In [ ]:
#카테고리, 기업명 합치기
company_data_df = pd.merge(category_company_df, company_number_df, how = 'left')
company_data_df.to_csv('c:/pywork/company_data.csv', encoding = 'euc-kr')

# 2. 각 기업별 재무정보 가져오기

## 1) 네이버 금융에서 재무제표 가져오기

In [ ]:
#가공된 한국거래소 제공 csv 파일 불러오기
temp = pd.read_csv('c:/pywork/company_number.csv', encoding = 'euc-kr')
company_num_temp = temp['기업코드'].to_list()

In [ ]:
#해당 홈페이지 내 기업실적분석 데이터 가져오기
#최근 연간 실적(2020년 12월 데이터)의 순이익률, ROE, 부채비율, PER 가져오기
# 주의) 메모리에 무리가 가지 않도록 나눠서 파일 가져오기
company_num_1 = company_num_temp[:500]
finacne_data_df_1 = pd.DataFrame()

for code in company_num_1:
    print(code)
    financial_statement_temp = pd.read_html('https://finance.naver.com/item/main.nhn?code='+code)
    try :
        temp_data = financial_statement_temp[3].T.iloc[3,:].reset_index(drop = True)
        finacne_data_df_1[code] = temp_data[[4,5,6,10]]
    except :
        pass
    print('완료')

In [ ]:
#재무제표 가공하기
financial_statement_temp = finacne_data_df_1.T
financial_statement = financial_statement_temp[financial_statement_temp[4].notnull()]
financial_statement.reset_index(inplace = True)
financial_statement.columns = ['기업코드', '순이익률', 'ROE', '부채비율', 'PER']

#재무제표 저장하기
financial_statement.to_csv('c:/pywork/financial_statement_1.csv', encoding = 'euc-kr')

In [ ]:
#파일 가져오기2
company_num_2 = company_num_temp[500:1000]
finacne_data_df_2 = pd.DataFrame()

for code in company_num_2:
    print(code)
    financial_statement_temp = pd.read_html('https://finance.naver.com/item/main.nhn?code='+code)
    try :
        temp_data = financial_statement_temp[3].T.iloc[3,:].reset_index(drop = True)
        finacne_data_df_2[code] = temp_data[[4,5,6,10]]
    except :
        pass
    print('완료')
    
financial_statement_temp = finacne_data_df_2.T
financial_statement = financial_statement_temp[financial_statement_temp[4].notnull()]
financial_statement.reset_index(inplace = True)
financial_statement.columns = ['기업코드', '순이익률', 'ROE', '부채비율', 'PER']
financial_statement.to_csv('c:/pywork/financial_statement_2.csv', encoding = 'euc-kr')

In [ ]:
#파일 가져오기3
company_num_3 = company_num_temp[1000:1500]
finacne_data_df_3 = pd.DataFrame()

for code in company_num_3:
    print(code)
    financial_statement_temp = pd.read_html('https://finance.naver.com/item/main.nhn?code='+code)
    try :
        temp_data = financial_statement_temp[3].T.iloc[3,:].reset_index(drop = True)
        finacne_data_df_3[code] = temp_data[[4,5,6,10]]
    except :
        pass
    print('완료')
    
financial_statement_temp = finacne_data_df_3.T
financial_statement = financial_statement_temp[financial_statement_temp[4].notnull()]
financial_statement.reset_index(inplace = True)
financial_statement.columns = ['기업코드', '순이익률', 'ROE', '부채비율', 'PER']
financial_statement.to_csv('c:/pywork/financial_statement_3.csv', encoding = 'euc-kr')

In [ ]:
#파일 가져오기4
company_num_4 = company_num_temp[1500:2000]
finacne_data_df_4 = pd.DataFrame()

for code in company_num_4:
    print(code)
    financial_statement_temp = pd.read_html('https://finance.naver.com/item/main.nhn?code='+code)
    try :
        temp_data = financial_statement_temp[3].T.iloc[3,:].reset_index(drop = True)
        finacne_data_df_4[code] = temp_data[[4,5,6,10]]
    except :
        pass
    print('완료')
    
financial_statement_temp = finacne_data_df_4.T
financial_statement = financial_statement_temp[financial_statement_temp[4].notnull()]
financial_statement.reset_index(inplace = True)
financial_statement.columns = ['기업코드', '순이익률', 'ROE', '부채비율', 'PER']
financial_statement.to_csv('c:/pywork/financial_statement_4.csv', encoding = 'euc-kr')

In [ ]:
#파일 가져오기5
company_num_5 = company_num_temp[2000:]
finacne_data_df_5 = pd.DataFrame()

for code in company_num_5:
    print(code)
    financial_statement_temp = pd.read_html('https://finance.naver.com/item/main.nhn?code='+code)
    try :
        temp_data = financial_statement_temp[3].T.iloc[3,:].reset_index(drop = True)
        finacne_data_df_5[code] = temp_data[[4,5,6,10]]
    except :
        pass
    print('완료')
    
financial_statement_temp = finacne_data_df_5.T
financial_statement = financial_statement_temp[financial_statement_temp[4].notnull()]
financial_statement.reset_index(inplace = True)
financial_statement.columns = ['기업코드', '순이익률', 'ROE', '부채비율', 'PER']
financial_statement.to_csv('c:/pywork/financial_statement_5.csv', encoding = 'euc-kr')

## 2) 재무제표 가공하기

In [ ]:
#데이터 불러오기
a = pd.read_csv('c:/pywork/financial_statement_1.csv', encoding = 'euc-kr')
b = pd.read_csv('c:/pywork/financial_statement_2.csv', encoding = 'euc-kr')
c = pd.read_csv('c:/pywork/financial_statement_3.csv', encoding = 'euc-kr')
d = pd.read_csv('c:/pywork/financial_statement_4.csv', encoding = 'euc-kr')
e = pd.read_csv('c:/pywork/financial_statement_5.csv', encoding = 'euc-kr')

In [ ]:
#데이터 합치기
financial_statement_df = pd.concat([a, b, c, d, e])

In [ ]:
#불필요한 데이터 제거하기
#재무제표 내에 빈칸이 있는 기업들 제거
columns = financial_statement_df.columns

for col in columns:
    financial_statement_df = financial_statement_df[financial_statement_df[col].notnull()]
    
#칼럼명 통일을 위해 칼럼명 변경
financial_statement_df = financial_statement_df[['기업코드', '순이익률', 'ROE', '부채비율', 'PER']]

#가공된 데이터 저장
financial_statement_df.to_csv('c:/pywork/financial_statement_data.csv', encoding = 'euc-kr')

# 3. 데이터 활용하기

## 1) 기업 카테고리 데이터와 기업 재무제표 데이터 합치기

In [ ]:
#데이터 불러오기
company_data_df = pd.read_csv('c:/pywork/company_data.csv', encoding = 'euc-kr')
finance_data_df = pd.read_csv('c:/pywork/financial_statement_data.csv', encoding = 'euc-kr')

In [ ]:
#데이터 가공하기
company_data_df = company_data_df[['카테고리', '기업명', '기업코드']]

#데이터 합치기
event_df = pd.merge(company_data_df, finance_data_df, how = 'inner')

#데이터 컬럼명 변경하기
event_df = event_df[['카테고리', '기업명', '기업코드', '순이익률', 'ROE', '부채비율', 'PER']]

#공백 데이터 제거하기
event_df = event_df[event_df.PER != '-']

#데이터 타입 통일하기
event_df = event_df.astype({'PER': 'float'})

#변경된 데이터 저장하기
event_df.to_csv('c:/pywork/event_df.csv', encoding = 'euc-kr')

## 2) 데이터 순위 매기기

In [ ]:
#데이터 불러오기
event_df = pd.read_csv('c:/pywork/event_df.csv', encoding = 'euc-kr')

In [ ]:
#필요한 칼럼별 순위 매기기
event_df['순이익률 순위'] = event_df['순이익률'].rank(ascending = False)
event_df['ROE 순위'] = event_df['ROE'].rank(ascending = False)
event_df['부채비율 순위'] = event_df['부채비율'].rank(ascending = True)
event_df['PER 순위'] = event_df['PER'].rank(ascending = True)

In [ ]:
#종합순위 매기기
event_df['순위합계'] = event_df['순이익률 순위'] + event_df['ROE 순위'] + event_df['부채비율 순위'] + event_df['PER 순위']
event_df['종합순위'] = event_df['순위합계'].rank()
event_rank_df = event_df.sort_values(ascending = True, by = '순위합계')

In [ ]:
#순위 데이터 저장하기
event_rank_df.to_csv('c:/pywork/event_rank_df.csv', encoding = 'euc-kr')